In [3]:
!pip install -U langchain-community weaviate-client langchain tiktoken pypdf rapidocr-onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: tiktoken
    Found existing installation: tiktoken 0.9.0
    Uninstalling tiktoken-0.9.0:
      Successfully uninstalled tiktoken-0.9.0


In [ ]:
!pip install sentence-transformers

In [ ]:
!pip install protobuf==3.20.0

In [ ]:
!pip install weaviate-client==3.25.0

In [ ]:
!pip install weaviate-client==3.24.1

In [ ]:
!pip install protobuf==3.20.0

In [ ]:
!pip install weaviate-client==3.23.1

In [5]:
WEAVIATE_API_KEY="MHVRbGczWmIvaEMyUjl2Ml83b2lhbGZ4bHlDbkdiWXZjYmZYMDF6ZEZ2MFRiV1BTWE1WU0RWYWRFZEpZPV92MjAw"
WEAVIATE_CLUSTER="https://qf9dqchbrakegqoejtdlw.c0.asia-southeast1.gcp.weaviate.cloud"

In [13]:
# Fixing Unicode Error in Google Colab
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [14]:
# Specify Embedding Model (Using Huggingface Sentence Transformer)
from langchain.embeddings import HuggingFaceEmbeddings
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)


/tmp/ipython-input-2276802976.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings

In [7]:
# Specify Embedding Model (Using Huggingface Sentence Transformer)
from langchain.embeddings import HuggingFaceEmbeddings
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)

 Loading the documents

In [9]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("/content/decease.pdf",extract_images=True)
pages = loader.load()

In [39]:
import os
import weaviate

WEAVIATE_URL = WEAVIATE_CLUSTER
WEAVIATE_API_KEY = WEAVIATE_API_KEY
client = weaviate.Client(
    url = WEAVIATE_URL,
    auth_client_secret = weaviate.AuthApiKey(api_key = WEAVIATE_API_KEY)
)

Backup code

from langchain.vectorstores import Weaviate
import weaviate

WEAVIATE_URL = WEAVIATE_CLUSTER
WEAVIATE_API_KEY = WEAVIATE_API_KEY
client = weaviate.Client(
    url = WEAVIATE_URL,
    auth_client_secret = weaviate.AuthApiKey(api_key = WEAVIATE_API_KEY)
)

Sentence Level Encoding/Embedding

Split the text into multiple documents

In [40]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 20)
docs = text_splitter.split_documents(pages)

In [41]:
from langchain.vectorstores import Weaviate
import weaviate
vector_db= Weaviate.from_documents(
                  docs, embeddings, client = client, weaviate_url=WEAVIATE_URL, by_text=False
              )

In [42]:
vector_db.similarity_search("what is AIDS related symptoms",k=3)

[Document(metadata={'creationdate': 'D:20250424095509', 'creator': 'PDFium', 'page': 99, 'page_label': '100', 'producer': 'PDFium', 'source': '/content/decease.pdf', 'total_pages': 118}, page_content='infection or due to a chronic illness such as rheumatoid arthritis; characterized by redness, \nswelling, stiffness, and pain  \nSyphilis- a sexually transmitted disease; initially causes only painless sores on the genitals \nbut can be life-threatening if untreated  \nSystemic- affecting the whole body  \nSystemic inflammatory response syndrome- a condition characterized by having two of \nthe following four clinical criteria: fever, rapid breathing, increased heart rate, and abnormal \nwhite blood cell count\n  \nSystemic lupus erythematosus- a disease of the immune system that causes \ninflammation of connective tissue in many areas of the body, including the skin, lungs, \nheart, joints, and kidneys  \nSystolic pressure- the blood pressure measured while the heart is contracting'),
 D

In [27]:
print(vector_db.similarity_search("what is AIDS related symptoms",k=3)[0].page_content)

infection or due to a chronic illness such as rheumatoid arthritis; characterized by redness, 
swelling, stiffness, and pain  
Syphilis- a sexually transmitted disease; initially causes only painless sores on the genitals 
but can be life-threatening if untreated  
Systemic- affecting the whole body  
Systemic inflammatory response syndrome- a condition characterized by having two of 
the following four clinical criteria: fever, rapid breathing, increased heart rate, and abnormal 
white blood cell count
  
Systemic lupus erythematosus- a disease of the immune system that causes 
inflammation of connective tissue in many areas of the body, including the skin, lungs, 
heart, joints, and kidneys  
Systolic pressure- the blood pressure measured while the heart is contracting


In [28]:
from langchain.prompts import ChatPromptTemplate

template = """You are an assistant for question-answering tasks.
            Use the following pieces of retrieved context to answer the question.
            If you don't know the answer, just say that you don't know.
            Use the sentence maximum and keep the answer concise.
            {context}
            Question: {question}
            Helpful Answer:"""


In [29]:
prompt = ChatPromptTemplate.from_template(template)

Load the Mistral LLM

In [ ]:
!pip install -U langchain-huggingface

In [67]:
from langchain_huggingface import HuggingFaceEndpoint

In [49]:
from google.colab import userdata
huggingfacehub_api_token= userdata.get('HUGGINGFACE_API_KEY')

In [68]:
from google.colab import userdata
huggingfacehub_api_token= userdata.get('HUGGINGFACE_API_KEY')

model = HuggingFaceEndpoint(
    huggingfacehub_api_token = huggingfacehub_api_token,
    repo_id = "mistralai/Mistral-7B-Instruct-v0.1",
    temperature = 1,
    max_length = 180
)

                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


In [56]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [57]:
output_parsers = StrOutputParser()
retriever = vector_db.as_retriever()

In [61]:
rag_chain = (
    {"context":retriever, "question":RunnablePassthrough()}
    | prompt
    | model
    | output_parsers
)

In [69]:
rag_chain.invoke("What is the sign of AIDS")

TypeError: InferenceClient.text_generation() got an unexpected keyword argument 'max_length'